In [101]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from collections import Counter
from seaborn import heatmap

import matplotlib.pyplot as plt
import spacy
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [102]:
from spacy.tokens import Doc, Token, Span

In [103]:
import gensim
import textacy

In [99]:
import en_core_web_lg
nlp= spacy.load('en_core_web_lg')

ValueError: Cannot create vectors table with dimension 0.
If you're using pre-trained vectors, are the vectors loaded?

In [24]:
df=pd.read_csv('clean_stories.csv', index_col = 0)

In [25]:
df.head()

,story_title,story_text,story_len,book_id,book_author,book_title
0,The Orchard'S Grandmother,I must ask you to go back more than two hundre...,23313,30720,Various,Happy Days for Boys and Girls
1,Rough,"He was a donkey, and we called him Rough. He b...",10413,30720,Various,Happy Days for Boys and Girls
2,The Music Lesson,"Touch the keys _lightly_, Nellie, my dear: ...",302,30720,Various,Happy Days for Boys and Girls
3,The Frost,"The frost looked forth one still clear night, ...",1469,30720,Various,Happy Days for Boys and Girls
4,My Picture,I have a little picture; Perchance you have...,1051,30720,Various,Happy Days for Boys and Girls


## Data_Cleaning (Move to Previous Notebook)

In [26]:
df['is_float']=df.story_text.apply(lambda x: isinstance(x, float))

In [27]:
df.drop(index=df[df.is_float==True].index, inplace=True)
df.reset_index(inplace=True, drop=True)

In [43]:
df = df[['story_title', 'story_text', 'book_id', 'book_author', 'book_title']]

In [29]:
df.drop(index = [9471, 9472], inplace= True)
df.reset_index(inplace=True, drop=True)

In [30]:
df['preface']=df.story_title.apply(lambda x: 'preface' in x.lower().split(' '))

In [31]:
df.drop(index = df[df.preface==True].index, inplace=True)

In [32]:
df.reset_index(inplace=True)

In [33]:
df.to_csv('story_data.csv')

In [104]:
df=pd.read_csv('story_data.csv', index_col='index')

In [105]:
df.head()

,Unnamed: 0,story_title,story_text,book_id,book_author,book_title,preface
index,,,,,,,
0,0,The Orchard'S Grandmother,I must ask you to go back more than two hundre...,30720,Various,Happy Days for Boys and Girls,False
1,1,Rough,"He was a donkey, and we called him Rough. He b...",30720,Various,Happy Days for Boys and Girls,False
2,2,The Music Lesson,"Touch the keys _lightly_, Nellie, my dear: ...",30720,Various,Happy Days for Boys and Girls,False
3,3,The Frost,"The frost looked forth one still clear night, ...",30720,Various,Happy Days for Boys and Girls,False
4,4,My Picture,I have a little picture; Perchance you have...,30720,Various,Happy Days for Boys and Girls,False


In [107]:
from random import randrange

In [113]:
index=randrange(len(df.index))
print(df.loc[index, 'story_title'])
print(df.loc[index, 'story_text'])

The Wife From Fairy-Land
Her talk was of all woodland things,    Of little lives that pass   Away in one green afternoon,    Deep in the haunted grass.   For she had come from fairy-land,    The morning of a day   When the world that still was April    Was turning into May.   Green leaves and silence and two eyes--    'Twas so she seemed to me;   A silver shadow of the woods,--    Whisper and mystery.   I looked into her woodland eyes,    And all my heart was hers;   And then I led her by the hand    Home up my marble stairs.   And all my granite and my gold    Was hers for her green eyes,   And all my sinful heart was hers,    From sunset to sunrise.   I gave her all delight and ease    That God had given to me,   I listened to fulfil her dreams,    Rapt with expectancy.   But all I gave and all I did    Brought but a weary smile   Of gratitude upon her face--    As though, a little while,   She loitered in magnificence    Of marble and of gold,   And waited to be home again,    When 

In [108]:
def word_count(string):
    return len(string.split(' '),'story_text')

In [100]:
def remove_underscores(string):
  split_string=string.split(' ')
  return ' '.join([word.rstrip('_').lstrip('_') for word in split_string])

In [46]:
df['word_count']= df.story_text.map(lambda x: word_count(x))

In [ ]:
new_df['clean_text'] = new_df.story_text.apply(lambda x: remove_underscores)

In [63]:
df.loc[15880, :]

story_title                               The Crimson Fairy Book
story_text     By Various  Edited by Andrew Lang   Preface Ea...
book_id                                                     2435
book_author                                         Lang, Andrew
book_title                                The Crimson Fairy Book
word_count                                                 98006
Name: 15880, dtype: object

In [74]:
new_df= df[(df.word_count>50)&(df.word_count<=10000)]
new_df.reset_index(inplace=True)

In [75]:
new_df.shape

(22399, 7)

In [77]:
df[df.word_count<50].shape

(40, 6)

In [ ]:
def word_count(string):
    return len(string.split(' '))

In [ ]:
df['clean_text'] = df.story_text.apply(lambda x: remove_underscores(x))

In [ ]:
df['word_count']= df.story_text.map(lambda x: word_count(x))

## EDA

### Spacy Preprocessing 

In [16]:
def check_stop_punct(token):
    if token.is_stop or token.is_punct:
        return False
    else:
        return True

def clean_doc_lem_stop(document):
    no_stop_li = list(filter(check_stop_punct, document))
    lemma_li = list(map(lambda x: x.lemma_, no_stop_li))
    return " ".join(lemma_li)

def remove_stop_lemmatize(text_series):
    """accepts a pandas series of strings and returns a lemmatized version
    that also drops stopwords.
    """
    doc_series = text_series.apply(nlp)
    # stopless is a list
    cleaned_series = doc_series.apply(clean_doc_lem_stop)
    return cleaned_series

In [17]:
def clean_doc_lem_stop(document):
    no_stop_li = list(filter(check_stop_punct, document))
    lemma_li = list(map(lambda x: x.lemma_, no_stop_li))
    return " ".join(lemma_li)

In [18]:
def remove_stop_lemmatize(text_series):
    """accepts a pandas series of strings and returns a lemmatized version
    that also drops stopwords.
    """
    doc_series = text_series.apply(nlp)
    # stopless is a list
    cleaned_series = doc_series.apply(clean_doc_lem_stop)
    return cleaned_series

In [21]:
df.head()

,index,story_title,story_text,book_id,book_author,book_title
0,0,The Orchard'S Grandmother,I must ask you to go back more than two hundre...,30720,Various,Happy Days for Boys and Girls
1,1,Rough,"He was a donkey, and we called him Rough. He b...",30720,Various,Happy Days for Boys and Girls
2,2,The Music Lesson,"Touch the keys _lightly_, Nellie, my dear: ...",30720,Various,Happy Days for Boys and Girls
3,3,The Frost,"The frost looked forth one still clear night, ...",30720,Various,Happy Days for Boys and Girls
4,4,My Picture,I have a little picture; Perchance you have...,30720,Various,Happy Days for Boys and Girls


#### NLP Pipe

In [80]:
new_df['char_len']=new_df.story_text.apply(lambda x: len(x))

/Users/antonhaugen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [91]:
new_df.loc[1646, :]

index                                                       1700
story_title                                       An Indian Tale
story_text     Nearly a hundred years ago, when the greater p...
book_id                                                    18907
book_author                                 Higgins, Emily Mayer
book_title     Holidays at the Grange; or, A Week's Delight\r...
word_count                                                  8937
char_len                                                   50135
Name: 1646, dtype: object

In [92]:
new_df[new_df.char_len<50000]

,index,story_title,story_text,book_id,book_author,book_title,word_count,char_len
0,0,The Orchard'S Grandmother,I must ask you to go back more than two hundre...,30720,Various,Happy Days for Boys and Girls,4438,23283
1,1,Rough,"He was a donkey, and we called him Rough. He b...",30720,Various,Happy Days for Boys and Girls,1975,10325
2,2,The Music Lesson,"Touch the keys _lightly_, Nellie, my dear: ...",30720,Various,Happy Days for Boys and Girls,77,302
3,3,The Frost,"The frost looked forth one still clear night, ...",30720,Various,Happy Days for Boys and Girls,343,1414
4,4,My Picture,I have a little picture; Perchance you have...,30720,Various,Happy Days for Boys and Girls,260,1051
...,...,...,...,...,...,...,...,...
22394,23103,Mortuary Customs,"1240. If ""salt water pigeons'"" feathers are in...",18992,frozenset(),Current Superstitions\nCollected from the Oral...,1170,5919
22395,23104,Bodily Affections,"1340. If the right cheek burns, some one is sp...",18992,frozenset(),Current Superstitions\nCollected from the Oral...,943,4284
22396,23105,Apparel,1376. If you mend your apron or dress while on...,18992,frozenset(),Current Superstitions\nCollected from the Oral...,867,3649
22397,23106,The Moon,"All cereals, when planted in the waxing of the...",18992,frozenset(),Current Superstitions\nCollected from the Oral...,2199,12492


In [78]:
list_of_docs= remove_stop_lemmatize(new_df.story_text)

ValueError: [E088] Text of length 1105005 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [93]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f8488a4de90>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f84aa373c20>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f84a97bebb0>)]